# View 3D images using napari

In [1]:
import napari
from tifffile import imread

In [2]:
## Load image
file = "../data/Kuett_2022/MainHer2BreastCancerModel/measured_mask_final_segmentation_hwatershed_500.00_90%.tif"
img = imread(file)

In [3]:
napari.view_image(img, scale=[2, 1, 1], ndisplay=3)

Viewer(camera=Camera(center=(151.0, 243.5, 325.5), zoom=2.573565573770492, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(150.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=3, last_used=0, range=((0.0, 304.0, 2.0), (0.0, 488.0, 1.0), (0.0, 652.0, 1.0)), current_step=(75, 243, 325), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'img' at 0x7bd275387950>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x7bd27beac180>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})